In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
import sqlite3
import ray
import matplotlib.pyplot as plt
from optuna.integration import LightGBMPruningCallback
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_normalizada_fe.parquet', engine='fastparquet')

In [3]:
# Variables predictoras y objetivo
# filtrar que en X el periodo sea menor o igual a 201910
# En x eliminar la columna 'CLASE_ZSCORE' y 'CLASE_DELTA_ZSCORE' si existen
cols_to_drop = [col for col in ['CLASE_ZSCORE', 'CLASE_DELTA_ZSCORE'] if col in df_full.columns]
X = df_full[df_full['PERIODO'] <= 201910].drop(columns=cols_to_drop)
# Filtrar en y que el periodo sea menor o igual a 201910 y que la columna exista
y = df_full[df_full['PERIODO'] <= 201910]['CLASE_DELTA_ZSCORE']
# Eliminar df_full para liberar memoria
del df_full
gc.collect()

48

In [4]:
# Definir los periodos de validación 
periodos_valid = [201909, 201910]

# Separar train y cinco conjuntos de validación respetando la secuencia temporal
X_train = X[X['PERIODO'] < periodos_valid[0]]
y_train = y[X['PERIODO'] < periodos_valid[0]]
X_val_list = [X[X['PERIODO'] == p] for p in periodos_valid]
y_val_list = [y[X['PERIODO'] == p] for p in periodos_valid]
del X, y
gc.collect()

0

In [5]:
from autogluon.tabular import TabularPredictor

# Dataset de entrenamiento
df_train = X_train.copy()
df_train['CLASE_DELTA_ZSCORE'] = y_train

# Dataset de validación
df_val = X_val_list[0].copy()
df_val['CLASE_DELTA_ZSCORE'] = y_val_list[0]

In [ ]:
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor(
    label='CLASE_DELTA_ZSCORE',
    problem_type='regression',
    eval_metric='mean_absolute_error'
).fit(
    train_data=df_train,
    tuning_data=df_val,
    presets='best_quality',
    time_limit= 25200,  # 7 horas máximo
    use_bag_holdout=True,
    hyperparameters={
        'GBM': [
            {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
            {'linear_tree': True, 'ag_args': {'name_suffix': 'LinearTree'}},
            {'num_leaves': 31, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'ag_args': {'name_suffix': 'Default'}}
        ],
        'XGB': [{}],
        'CAT': [{}],
        'NN_TORCH': [{
            'num_epochs': 20,           # controla duración
            'learning_rate': 1e-3,
            'ag_args': {'name_suffix': 'DL'},
        }]
    },
    ag_args_ensemble={
        'fold_fitting_strategy': 'sequential_local'  # mejora uso de memoria
    }
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250628_050549"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.23
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Apr 17 19:20:47 UTC 2
CPU Count:          28
Memory Avail:       95.35 GB / 125.58 GB (75.9%)
Disk Space Avail:   328.07 GB / 543.17 GB (60.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to False. Reason: Skip dynamic_stacking when use_bag_holdout is enabled. (use_bag_holdout=True)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 25200s
AutoGluon will save models to "/home/pablo/Documentos/labo3-2025v/AutogluonModels/ag-20250628_050549"
Train Data Rows:    14804223
Train Data Columns: 178
Tuning Data Rows:    541842
Tuning Data Columns: 178
L

[1000]	valid_set's l1: 0.252046
[2000]	valid_set's l1: 0.249979
[3000]	valid_set's l1: 0.248599


In [ ]:
# predictor = TabularPredictor.load("AutogluonModels/ag-20250627_202659")

# predictor.fit_extra(
#     hyperparameters='default',
#     time_limit=32400,
#     ag_args_fit={"ag.max_memory_usage_ratio": 2.0},
#     excluded_model_types=["KNN", "NN_TORCH", "CATBOOST"]
# )


In [ ]:

# Comparar MAE y MedAE en el leaderboard
lb = predictor.leaderboard(data=df_val,extra_metrics=['mean_absolute_error', 'median_absolute_error'], silent=False)

In [ ]:
# Evaluar feature importance sobre el conjunto de validación
importancia = predictor.feature_importance(df_val)

In [ ]:
# Combinar entrenamiento + validación
df_full = pd.concat([df_train, df_val], axis=0)
del df_train, df_val
gc.collect()

In [ ]:
# Reentrenar el mejor modelo con TODOS los datos disponibles
predictor_full = predictor.refit_full(train_data=df_full)

In [ ]:
# Verificar los modelos disponibles (el mejor ahora tiene el sufijo '_FULL')
print("Modelos disponibles luego del refit completo:")
print(predictor.leaderboard(silent=True)['model'].tolist())
# Eliminar modelos intermedios para liberar espacio
predictor.delete_models(models_to_keep='best', dry_run=False)

# Confirmar que solo queda el modelo reentrenado
print("\nModelos restantes después de eliminar los intermedios:")
print(predictor.leaderboard(silent=True)['model'].tolist())

# (Opcional) Guardar el predictor final si querés usarlo luego sin volver a cargar todo
predictor.save('./data/modelo_final_autogluon')

# ---  Liberar memoria ---
del df_full
gc.collect()


In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones
df_pred_full = pd.read_parquet('./data/l_vm_completa_normalizada_fe.parquet', engine='fastparquet')
# Dejo solo los datos del periodo 201910 y que A_PREDECIR sea True
# Filtrar solo los datos del periodo 201910 y donde A_PREDECIR sea True
df_pred_full = df_pred_full[
    (df_pred_full['PERIODO'] == 201910) & (df_pred_full['A_PREDECIR'] == True)
].drop(columns=['CLASE_ZSCORE', 'CLASE_DELTA_ZSCORE'])

In [ ]:
# Realizar las predicciones usando el predictor original
predictions = predictor.predict(df_pred_full)
# Agregar las predicciones al DataFrame original
df_pred_full['CLASE_DELTA_ZSCORE'] = predictions

In [ ]:
# Imprimir la lista de columas del DataFrame con las predicciones
print("Columnas del DataFrame con las predicciones:")
print(df_pred_full.columns.tolist())

In [ ]:
# Dernormalizar la columna CLASE_DELTA_ZSCORE
df_pred_full['CLASE_DELTA'] = df_pred_full['CLASE_DELTA_ZSCORE'] * df_pred_full['CLASE_DELTA_STD'] + df_pred_full['CLASE_DELTA_MEAN']
df_pred_full['TN'] = df_pred_full['TN_ZSCORE'] * df_pred_full['TN_STD'] + df_pred_full['TN_MEAN']
# Agregar la columna TN_PREDICT que sea la suma de TN y CLASE_DELTA y si es menor que cero, poner cero
df_pred_full['TN_PREDICT'] = df_pred_full['TN'] + df_pred_full['CLASE_DELTA']
df_pred_full['TN_PREDICT'] = df_pred_full['TN_PREDICT'].clip(lower=0)

In [ ]:
# Generar Dataframe que contenga por cada PRODUCT_ID la suma de TN_PREDICT
df_final = df_pred_full.groupby('PRODUCT_ID').agg({'TN_PREDICT': 'sum'}).reset_index()
df_final = df_final.rename(columns={'PRODUCT_ID': 'product_id', 'TN_PREDICT': 'tn'})
# Guardar el DataFrame df_final en un archivo CSV
df_final.to_csv('./modelos/autoglun_normalizando_clase_delta.csv', index=False)
df_final.shape
